# 04. LC Extraction and Visualization

This notebook extracts quantitative R2* metrics from the LC ROI and visualizes the results.

In [ ]:
import sys
sys.path.append('../')
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from src.io import load_nifti
from src.extraction import extract_roi_stats

ATLAS_DIR = '../atlases'
OUTPUT_DIR = '../outputs/results'
FIGURES_DIR = '../outputs/figures'

## 1. Extract Metrics

Calculate mean R2* within the LC atlas for each subject.

In [ ]:
atlas_filename = 'LC_atlas_MNI.nii.gz'
atlas_path = os.path.join(ATLAS_DIR, atlas_filename)

results = []

if os.path.exists(atlas_path):
    lc_atlas = load_nifti(atlas_path)
    
    # Iterate through processed subjects
    # Note: In a real scenario, we'd loop through all subjects in data/
    # Here we check for processed outputs
    processed_files = [f for f in os.listdir(OUTPUT_DIR) if f.endswith('_R2star_MNI.nii.gz')]
    
    for f in processed_files:
        sub_id = f.split('_')[0]
        r2s_path = os.path.join(OUTPUT_DIR, f)
        r2s_img = load_nifti(r2s_path)
        
        print(f"Extracting from {sub_id}...")
        stats = extract_roi_stats(r2s_img, lc_atlas)
        stats['subject_id'] = sub_id
        results.append(stats)
        
    df = pd.DataFrame(results)
    print(df)
    
    # Save results
    df.to_csv(os.path.join(OUTPUT_DIR, 'lc_metrics.csv'), index=False)
else:
    print("Atlas not found.")

## 2. Visualization

Plot the distribution of LC R2* values.

In [ ]:
if not df.empty:
    plt.figure(figsize=(8, 6))
    sns.boxplot(y=df['mean'])
    plt.title("LC Mean R2* Distribution")
    plt.ylabel("R2* (1/s)")
    
    output_plot = os.path.join(FIGURES_DIR, 'LC_R2star_Boxplot.png')
    plt.savefig(output_plot)
    plt.show()
    print(f"Plot saved to {output_plot}")